# Assignment 1
**Emma McCready**

---

### Analysis of population over time

You are required to collect, process, analyse and interpret the data in order to identify possible issues/problems at present and make predictions/classifications in regard to the future. This analysis will rely on the available data from CSO and any additional data you deem necessary (with supporting evidence) to support your hypothesis for this scenario.

Areas of focus:
* Annual Population Change
* Population Forecasting

To do:

* Re organise this sheet into Data Cleaning, then Data prep (if this is the correct order?), then EDA... and so on
* Maybe write a function to automatically prepare the data for each type of migration?
* I'd rather if net migration included the thousands in it
* Investigate "All countries" under countries dataset

--- 

- diagnosing the “tidiness” of the data — how much data cleaning we will have to do
- reshaping the data — getting right rows and columns for effective analysis
- combining multiple files
- changing the types of values — how we fix a column where numerical values are stored as strings, for example
- dropping or filling missing values - how we deal with data that is incomplete or missing
- manipulating strings to represent the data better

In [1]:
# Dependencies.. put any packages I install here:
#!pip install flask pandas==2.1.2

In [2]:
# Load in packages
import pandas as pd
import numpy as np
from scipy import stats


# Part one: Data Inspection

<details>
    <summary style="display:list-item; font-size:16px; color:blue;"><i>How did I approach this and why did I chose this approach?</i></summary>

include details..
</details>

In [3]:
# Load in data

pop_data = pd.read_csv("migration_data.csv")

#printing the first 5 rows
pop_data.head()

,STATISTIC Label,Year,Country,Sex,Origin or Destination,UNIT,VALUE
0,Estimated Migration (Persons in April),1987,United Kingdom (1),Both sexes,Net migration,Thousand,-13.7
1,Estimated Migration (Persons in April),1987,United Kingdom (1),Both sexes,Emigrants: All destinations,Thousand,21.8
2,Estimated Migration (Persons in April),1987,United Kingdom (1),Both sexes,Immigrants: All origins,Thousand,8.1
3,Estimated Migration (Persons in April),1987,United Kingdom (1),Male,Net migration,Thousand,-9.0
4,Estimated Migration (Persons in April),1987,United Kingdom (1),Male,Emigrants: All destinations,Thousand,13.1


In [4]:
# Making sure no duplicates:
duplicates = pop_data.duplicated()
print(duplicates.value_counts())  

False    2664
Name: count, dtype: int64


### thoughts on the above:

the dataframe is already in a good format, in that it has each variable as a separate column, and each row as a separate observation. So, no need for pd_melt() on first look. 
* The statistic label column seems a bit redundant though, and it would be more favourable to remove the unit column and instead have the VALUE column reflect it instead (ie. multiply it by 1000).
* Will need to confirm the vairbales are stored as the correct data type.
* no duplicates which is good

### Getting a sense of the data...

In [5]:
# messing around and mistyped this excluding brackets; this doesn't really tell me anything new but does show me first 5 and last 5 rows:
#pop_data.info

#what I was meant to type:
pop_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2664 entries, 0 to 2663
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   STATISTIC Label        2664 non-null   object 
 1   Year                   2664 non-null   int64  
 2   Country                2664 non-null   object 
 3   Sex                    2664 non-null   object 
 4   Origin or Destination  2664 non-null   object 
 5   UNIT                   2664 non-null   object 
 6   VALUE                  2104 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 145.8+ KB


In [6]:
# getting numerical summaries

pop_data.describe()

,Year,VALUE
count,2664.000000,2104.000000
mean,2005.000000,8.943726
std,10.679083,15.513703
min,1987.000000,-43.900000
25%,1996.000000,1.800000
50%,2005.000000,4.700000
75%,2014.000000,10.200000
max,2023.000000,151.100000


<summary style="color:blue;">
    
Notes on the above:
    
* Shape: there's 2664 rows, and 7 columns in the dataset (no need to use `.shape`, as I got this from `.info()`). 
* the "year" column and the "VALUE" column are the only numerical variables. Year should be a categorical variable.
* From min/max of the Year col, I can see that the population data ranges from the years 1987 to 2023. 
* Regarding the value column, I note from when I called `pop_data.head()` that this is variable contains information on the net migration, as well as the number of incoming and outgoing people all as individual observations. So, the statistical information extracted (ie. the mean, std, etc.) isn't reliable.  
* Also worth noting is the difference between the counts for "Year" and "VALUE". There's a lot more values for Year, implying missing data in VALUE. So I would like to find out just how many are empty, and may as well check the whole dataset for missing data:

In [7]:
# find whether columns contain null values
print(pop_data.isnull().sum())

STATISTIC Label            0
Year                       0
Country                    0
Sex                        0
Origin or Destination      0
UNIT                       0
VALUE                    560
dtype: int64


In [8]:
# Summarising the categorical variables:
pop_data.describe(include=object)

,STATISTIC Label,Country,Sex,Origin or Destination,UNIT
count,2664,2664,2664,2664,2664
unique,1,8,3,3,1
top,Estimated Migration (Persons in April),United Kingdom (1),Both sexes,Net migration,Thousand
freq,2664,333,888,888,2664


<summary style="color:blue;">
    
Not much valuable info above I think? But I'm curious about the modes for each of these, I assume they are just listed in as the mode as they are the top value in the dataset, but just for piece of mind:

In [9]:
print('Value counts for "Country":\n', pop_data['Country'].value_counts(), '\n\nValue counts for "sex":\n', pop_data['Sex'].value_counts())

# I dislike these labels for countries - may have to amend and change to UK, US, Canada, Australia, Other, All countries, EU14 (Excl UK, IE), EU 15-27,



Value counts for "Country":
 Country
United Kingdom (1)                                     333
United States                                          333
Canada                                                 333
Australia                                              333
Other countries (23)                                   333
All countries                                          333
EU14 excl Irl (UK & Ireland)                           333
EU15 to EU27 (accession countries joined post 2004)    333
Name: count, dtype: int64 

Value counts for "sex":
 Sex
Both sexes    888
Male          888
Female        888
Name: count, dtype: int64


<summary style="color:blue;">
Concerned about "All countries" under the Country column, is it a total? 

In [20]:
#failing below, try fix
#if pop_data.Year == 2020:
    #print(pop_data.Country, pop_data.VALUE)

    
##  another fail


#print(pop_data['Year'] == 2020)
#data_2020 = pop_data['Year'] in 2020

    
##  another fail



#for year in pop_data['Year']:
    #if year == 2020:
        #print(pop_data['Country'], pop_data['VALUE'])
    
    
##  another fail but .loc[] is probs what I'm looking for

#pop_data.loc[2020:2021, 'Country':'VALUE']
#pop_data.loc[pop_data['Year' == 2020]]
#pop_data.loc['Year', 2020]


KeyError: 2020

In [11]:
# Obtaining some further information on the dataset:
pop_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2664 entries, 0 to 2663
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   STATISTIC Label        2664 non-null   object 
 1   Year                   2664 non-null   int64  
 2   Country                2664 non-null   object 
 3   Sex                    2664 non-null   object 
 4   Origin or Destination  2664 non-null   object 
 5   UNIT                   2664 non-null   object 
 6   VALUE                  2104 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 145.8+ KB


<summary style="color:blue;">
Initial thoughts: This dataset is a count of the total number of emigrants and immigrants in any given year, to and from a given county.

The first column is a bit redundant and it's a bit inconvenient to have the net migration plus the number of immigrants and emigrants in the same column I think, but I should be able to find a work around without making additional unnecessary columns.

<summary style="color:blue;">

### Tiding it up a bit to make it easier to work with:
Ideas for tidying:
* make some things lower case?
* rename cols, e.g. instead of "Origin or Destination" change it to migration_type

In [12]:
# making sure the there is only 3 types of "Origin or Destination"
#print(pop_data['Origin or Destination'].value_counts())          # output confirms this 

In [13]:
# Making it lowercase
## acc no dont do this

#pop_data = pop_data['migration_type'].str.lower() #.str is necessary as it's a string

# Dropping the unecessary column, STATISTIC Label, as it doesn't contain useful values. I want to make the dataset easier to visually explore.
pop_data = pop_data.drop(columns = ['STATISTIC Label', 'UNIT'])

# Renaming columns and making them lowercase for convenience
pop_data = pop_data.rename(columns = {'Year':'year', 'Country':'country', 'Sex':'sex', 'Origin or Destination':'migration_type', 'VALUE':'total_migration'})
pop_data['total_migration'] = (pop_data['total_migration'] * 1000)
pop_data['total_migration'] = pop_data['total_migration'].round(decimals=0)
#Rename countries?


#pop_data.head()

In [14]:
## Goal: Extract all instances of net migration and move it into a new dataframe. Make sure to bring all other cols with it.

In [15]:
#loc to pull out the info I want
#net_migration = pop_data.loc[["Net migration"], ["Origin or Destination"]]

## this is using = a boolean mask...
net_migration = (pop_data["migration_type"] == "Net migration")  # assigns each row w True or False based on whether or not the data is 'Net migration
both_sexes = (pop_data["sex"] == "Both sexes") # Same as above, but for both sexes
net_migration = pop_data[net_migration & both_sexes]# creates a new dataframe with just rows where net_migration was true for both sexes

#net_migration.head()

# Creating the opposite dataset.. figure this out?
#not_net_migration = ~pop_data[net_migration]
#individual_sexes = ~both_sexes
#migration_by_sex = not_net_migration & individual_sexes #pop_data[~net_migration & ~both_sexes]

Now the statistical info for the migration will be much more meaningful;

In [16]:
net_migration["total_migration"].describe()

count       236.000000
mean       5311.016949
std       17635.191046
min      -43900.000000
25%       -1400.000000
50%        1550.000000
75%        7250.000000
max      104800.000000
Name: total_migration, dtype: float64

In [17]:
# The country is categorical, as it's a finite list, so should amend that:
pop_data['country'] = pop_data['country'].astype('category')

# same for the year: 

pop_data['year'] = pop_data['year'].astype('category')

In [18]:
# up to date look at the data..
pop_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2664 entries, 0 to 2663
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   year             2664 non-null   category
 1   country          2664 non-null   category
 2   sex              2664 non-null   object  
 3   migration_type   2664 non-null   object  
 4   total_migration  2104 non-null   float64 
dtypes: category(2), float64(1), object(2)
memory usage: 69.5+ KB


In [19]:
# Should expect to see only statistics for the total column now, and not year anymore:
pop_data.describe()

,total_migration
count,2104.000000
mean,8943.726236
std,15513.702887
min,-43900.000000
25%,1800.000000
50%,4700.000000
75%,10200.000000
max,151100.000000


<summary style="color:blue;">

#### However, there's still the issue of missing values.
This data is likely missing due to systematic causes, i.e. the data was never provided in the first place. This data is missing at random (MAR), as there's no real logic or explanation other than it's generally older data that seems to be missing.
* For this reason, it might be appropriate to delete all of the rows containing empty values, OR to limit the range of the analysis (e.g. limit it from 2010 -> present)
* It could be considered to try to insert the mean of the values above and below each missing value. Will have to investigate and decide

In [20]:
pop_data.head()

,year,country,sex,migration_type,total_migration
0,1987,United Kingdom (1),Both sexes,Net migration,-13700.0
1,1987,United Kingdom (1),Both sexes,Emigrants: All destinations,21800.0
2,1987,United Kingdom (1),Both sexes,Immigrants: All origins,8100.0
3,1987,United Kingdom (1),Male,Net migration,-9000.0
4,1987,United Kingdom (1),Male,Emigrants: All destinations,13100.0


In [21]:
net_migration.head()

,year,country,sex,migration_type,total_migration
0,1987,United Kingdom (1),Both sexes,Net migration,-13700.0
9,1987,United States,Both sexes,Net migration,-6900.0
18,1987,Canada,Both sexes,Net migration,NaN
27,1987,Australia,Both sexes,Net migration,NaN
36,1987,Other countries (23),Both sexes,Net migration,-1400.0
